## Perhitungan Nilai Pengaturan Arduino UNO Terbaik Untuk Tracking Teleskop Ganda Zeiss

Oleh    : Irfan Imaduddin <br>
Tanggal : 29 Oktober 2020 <br>
Ref     : Moch. Irfan, 2020 <br>
Versi   : 1.0 <br>
Kontak  : irfanimaduddin@gmail.com <br>

---

1. Import modul yang digunakan

In [5]:
import numpy as np

Dari buku Astronomical Almanac 2020 halaman B9, diketahui bahwa pada tahun 2020,

$$\bar t_{sidereal}\ (s) = 0.997 269 566 32\ \bar t_{solar}\ (s)$$

Spesifikasi perhitungan (patokan Arduino UNO):
- $\Delta t = 62.5$ ns (Resolusi waktu Arduino UNO dengan crystal clock 16 MHz)
- Prescale $= 2^3 - 2^{10}$
- Count $= 16$ bit $= 65536$ count

Spesifikasi perhitungan (tambahan info dari Teleskop Zeiss dan kamera ZWO ASI 224MM):
- $F_{teleskop} \sim 11000$ mm (Panjang fokus Teleskop Zeiss)
- $pix_{size} = 3.75\ \mu$m (Ukuran piksel kamera)

Challenge: Menghitung nilai prescale dan count terdekat serta pergeseran <i>tracking</i> dalam piksel per jam

2. Definisikan fungsi untuk menentukan nilai prescaler dan count yang menghasilkan nilai perhitungan terdekat dengan nilai yang diinginkan

In [32]:
def find_nearest_val(value, timer=1): 
    
    # Define initial values
    res_t     = 62.5 # Time resolution of UNO's crystal clock (nanoseconds) 
    f_tele    = 11000 # mm
    p_size    = 3.75 # micron
    
    # Define nearest prescale and count indices 
    near_scale, near_count = 0, 0

    # Declaring a variable to store the minimum absolute difference 
    delta   = float('inf') 
    
    if timer==0:
        # Pre scaler values
        pre_scale = np.array([1, 8, 64, 256, 1024])
        
        # Acceptable count value
        count_1   = np.arange(1, 257, 1)
        count_2   = np.arange(1, 1025, 1)
        
        for i in range(len(pre_scale)):
            for j in range(len(count_1)):
                for k in range(len(count_2)):
                    # Finding the value
                    time = res_t * pre_scale[i] * count_1[j] * count_2[k] / 1e6

                    # Updating the index of the answer if new absolute difference is less than delta
                    if abs(time - value) < delta: 
                        delta = abs(time - value) 
                        near_scale, near_count_1, near_count_2 = i, j, k

        # Calculate pixel scale
        p_scale = 206265. * p_size / f_tele / 1000. # arcseconds per pixel

        # Calculate nearest value (miliseconds)
        result = res_t * pre_scale[near_scale] * count_1[near_count_1]  * count_2[near_count_2] / 1e6

        # Calculate error of calculated over required values (percent)
        error = delta / value * 100.

        # Calculate error of calculated over required values (arcseconds per hour)
        err_angle = 360. * 3600. * error / 100. / 24

        # Calculate error of calculated over required values (pixels per hour)
        err_pixel = err_angle / p_scale

        param = [timer, pre_scale[near_scale], count_1[near_count_1], count_2[near_count_2], result, delta, error, err_angle, err_pixel]

        print("Value to be reached = %.8f ms \n" %(value))
        print("Timer = %i" %(timer))
        print("\nResult!")
        print("Nearest prescaler = %i" %(pre_scale[near_scale]))
        print("Nearest count = %i" %(count_1[near_count_1]))
        print("Looping = %i" %(count_2[near_count_2]))
        print("Nearest value with 16MHz clock = %.8f ms" %(result))
        print("Difference of required and calculated time = %.8f ms" %(delta))
        print("Difference ratio = %.5f percent" %(error))
        print("Difference rate = %.5f arcsec per hour" %(err_angle))
        print("Difference rate = %.5f pixels per hour" %(err_pixel))

    elif timer==1:
        # Pre scaler values
        pre_scale = np.array([1, 8, 64, 256, 1024])

        # Acceptable count value
        count   = np.arange(1, 65537, 1)
        
        for i in range(len(pre_scale)):
            for j in range(len(count)):
                # Finding the value
                time = res_t * pre_scale[i] * count[j] / 1e6

                # Updating the index of the answer if new absolute difference is less than delta
                if abs(time - value) < delta: 
                    delta = abs(time - value) 
                    near_scale, near_count = i, j

        # Calculate pixel scale
        p_scale = 206265. * p_size / f_tele / 1000. # arcseconds per pixel

        # Calculate nearest value (miliseconds)
        result = res_t * pre_scale[near_scale] * count[near_count] / 1e6

        # Calculate error of calculated over required values (percent)
        error = delta / value * 100.

        # Calculate error of calculated over required values (arcseconds per hour)
        err_angle = 360. * 3600. * error / 100. / 24

        # Calculate error of calculated over required values (pixels per hour)
        err_pixel = err_angle / p_scale

        param = [timer, pre_scale[near_scale], count[near_count], result, delta, error, err_angle, err_pixel]

        print("Value to be reached = %.8f ms \n" %(value))
        print("Timer = %i" %(timer))
        print("\nResult!")
        print("Nearest prescaler = %i" %(pre_scale[near_scale]))
        print("Nearest count = %i" %(count[near_count]))
        print("Nearest value with 16MHz clock = %.8f ms" %(result))
        print("Difference of required and calculated time = %.8f ms" %(delta))
        print("Difference ratio = %.5f percent" %(error))
        print("Difference rate = %.5f arcsec per hour" %(err_angle))
        print("Difference rate = %.5f pixels per hour" %(err_pixel))

                    
    elif timer==2:
        # Pre scaler values
        pre_scale = np.array([1, 8, 32, 64, 128, 256, 1024])
        
        # Acceptable count value
        count_1   = np.arange(1, 257, 1)
        count_2   = np.arange(1, 1025, 1)
        
        for i in range(len(pre_scale)):
            for j in range(len(count_1)):
                for k in range(len(count_2)):
                    # Finding the value
                    time = res_t * pre_scale[i] * count_1[j] * count_2[k] / 1e6

                    # Updating the index of the answer if new absolute difference is less than delta
                    if abs(time - value) < delta: 
                        delta = abs(time - value) 
                        near_scale, near_count_1, near_count_2 = i, j, k

        # Calculate pixel scale
        p_scale = 206265. * p_size / f_tele / 1000. # arcseconds per pixel

        # Calculate nearest value (miliseconds)
        result = res_t * pre_scale[near_scale] * count_1[near_count_1]  * count_2[near_count_2] / 1e6

        # Calculate error of calculated over required values (percent)
        error = delta / value * 100.

        # Calculate error of calculated over required values (arcseconds per hour)
        err_angle = 360. * 3600. * error / 100. / 24

        # Calculate error of calculated over required values (pixels per hour)
        err_pixel = err_angle / p_scale

        param = [timer, pre_scale[near_scale], count_1[near_count_1], count_2[near_count_2], result, delta, error, err_angle, err_pixel]

        print("Value to be reached = %.8f ms \n" %(value))
        print("Timer = %i" %(timer))
        print("\nResult!")
        print("Nearest prescaler = %i" %(pre_scale[near_scale]))
        print("Nearest count = %i" %(count_1[near_count_1]))
        print("Looping = %i" %(count_2[near_count_2]))
        print("Nearest value with 16MHz clock = %.8f ms" %(result))
        print("Difference of required and calculated time = %.8f ms" %(delta))
        print("Difference ratio = %.5f percent" %(error))
        print("Difference rate = %.5f arcsec per hour" %(err_angle))
        print("Difference rate = %.5f pixels per hour" %(err_pixel))

    else:
        raise ValueError("Salah input nilai Timer!")
        
    return param

3. Definisikan nilai yang ingin dicapai

In [37]:
sidereal = 1994.53913264/2.  # miliseconds
mars     = 996.7291988 # miliseconds

4. Menghitung nilai yang ingin dicapai (sideris)

In [38]:
sidereal_res = find_nearest_val(sidereal, timer=0)

Value to be reached = 997.26956632 ms 

Timer = 0

Result!
Nearest prescaler = 256
Nearest count = 157
Looping = 397
Nearest value with 16MHz clock = 997.26400000 ms
Difference of required and calculated time = 0.00556632 ms
Difference ratio = 0.00056 percent
Difference rate = 0.30140 arcsec per hour
Difference rate = 4.28633 pixels per hour


In [39]:
sidereal_res = find_nearest_val(sidereal, timer=1)

Value to be reached = 997.26956632 ms 

Timer = 1

Result!
Nearest prescaler = 256
Nearest count = 62329
Nearest value with 16MHz clock = 997.26400000 ms
Difference of required and calculated time = 0.00556632 ms
Difference ratio = 0.00056 percent
Difference rate = 0.30140 arcsec per hour
Difference rate = 4.28633 pixels per hour


In [40]:
sidereal_res = find_nearest_val(sidereal, timer=2)

Value to be reached = 997.26956632 ms 

Timer = 2

Result!
Nearest prescaler = 128
Nearest count = 171
Looping = 729
Nearest value with 16MHz clock = 997.27200000 ms
Difference of required and calculated time = 0.00243368 ms
Difference ratio = 0.00024 percent
Difference rate = 0.13178 arcsec per hour
Difference rate = 1.87405 pixels per hour


5. Menghitung nilai yang ingin dicapai (mars)

In [17]:
mars_res = find_nearest_val(mars, timer=1)

Value to be reached = 1993.45839760 ms 

Timer = 1

Result!
Nearest prescaler = 1024
Nearest count = 31148
Nearest value with 16MHz clock = 1993.47200000 ms
Difference of required and calculated time = 0.01360240 ms
Difference ratio = 0.00068 percent
Difference rate = 0.36847 arcsec per hour
Difference rate = 5.24008 pixels per hour
